# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

import joblib

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

from azureml.core.webservice import LocalWebservice, AciWebservice
from azureml.core.model import Model
import requests
import json

from azureml.core import ScriptRunConfig

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'ml-experiment-2'

hyper_experiment=Experiment(ws, experiment_name)

hyper_experiment

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code R2Y6K6GP6 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


Name,Workspace,Report Page,Docs Page
ml-experiment-2,quick-starts-ws-132250,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster

amlcompute_cluster_name = "notebook132250"

# Verify that cluster does not exist already

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.

Running


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [8]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    '--C': choice(0.5, 1.5),
    'max_iter': choice( 50, 150)
})

#TODO: Create your estimator and hyperdrive config
# estimator = SKLearn(
#     source_directory=".",
#     compute_target=amlcompute_cluster_name,
#     entry_script="train.py",
#     script_params={'--C':1.0, '--max_iter':100})

# Create the environment
sklearn_env = Environment.get(workspace=ws, name="AzureML-Tutorial")

src = ScriptRunConfig(source_directory='./',
                      script='train.py',
                      compute_target = compute_target,
                      environment=sklearn_env)
                    

if "training" not in os.listdir():
    os.mkdir("./training")

In [9]:
hyperdrive_run_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name = "Accuracy",
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
    max_total_runs = 100,
    max_concurrent_runs = 2)

In [10]:
#TODO: Submit your experiment
hyperdrive_run = hyper_experiment.submit(config=hyperdrive_run_config, show_output=True)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
## Take SS

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_bb66effd-c5e1-4a39-9f10-54b3baef8e62
Web View: https://ml.azure.com/experiments/ml-experiment-2/runs/HD_bb66effd-c5e1-4a39-9f10-54b3baef8e62?wsid=/subscriptions/2c48c51c-bd47-40d4-abbe-fb8eabd19c8c/resourcegroups/aml-quickstarts-132250/workspaces/quick-starts-ws-132250

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-29T01:31:05.624081][API][INFO]Experiment created<END>\n"<START>[2020-12-29T01:31:08.0398717Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-12-29T01:31:08.602042][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space<END>\n""<START>[2020-12-29T01:31:08.911916][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_bb66effd-c5e1-4a39-9f10-54b3baef8e62
Web View: https://ml.azure.com/experiments/ml-experiment-2/runs/HD_bb66effd-c5e1-4a39-9f10-54b3baef8e62?wsid=/subscriptions/2c48c

{'runId': 'HD_bb66effd-c5e1-4a39-9f10-54b3baef8e62',
 'target': 'notebook132250',
 'status': 'Canceled',
 'startTimeUtc': '2020-12-29T01:31:05.383414Z',
 'endTimeUtc': '2020-12-29T01:35:43.597331Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name": 

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
## Take SS

In [ ]:
#TODO: Save the best model
print(best_run.get_file_names())
model = best_run.register_model(model_name = "Best", model_path=".")
joblib.dump(model, "models/hd_hearth.pkl")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:

# conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
# myenv.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
# myenv.python.conda_dependencies.add_pip_package("joblib")

# myenv.docker.base_image = None
# myenv.docker.base_dockerfile = "FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04\nRUN echo \"this is test\""
# myenv.inferencing_stack_version = "latest"

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)
# Define Deployment

deployment_config = LocalWebservice.deploy_configuration() 

service = Model.deploy(ws, 'myservice', [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)
print("scoring URI: " + service.scoring_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
scoring_uri = 'scoring uri for your service'
headers = {'Content-Type':'application/json'}

test_data = json.dumps({'text': 'Today is a great day!'})

response = requests.post(scoring_uri, data=test_data, headers=headers)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:

print(response.status_code)
print(response.elapsed)
print(response.json())

service.delete()